In [1]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import evaluate
import os
import json
import pandas as pd

from sklearn.model_selection import train_test_split 


 <font color="red">NOTE</font> This code is outdated and a new model shall be created. Culture_Coding.xlsx files have been changed to _Altogether_Dataset_RACoded.xlsx

In [ ]:
from huggingface_hub import notebook_login
# copy and paste this code in the terminal: huggingface-cli login 
# then paste your token


notebook_login()

## Import the dataset

In [2]:
df = pd.read_excel("../../RA_Cleaning/Culture_Coding.xlsx", header=[0,1], index_col=0)
df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: '../../RA_Cleaning/Culture_Coding.xlsx'

In [ ]:


# subdivide into just passage and outcome
df_small = pd.DataFrame()
df_small[["ID","passage","EVENT","CAUSE","ACTION"]] = df[[('CULTURE', "Passage Number"), ('CULTURE', "Passage"), ('EVENT', "No_Info"), ('CAUSE', "No_Info"), ('ACTION', "No_Info")]]
# Flip the lable of "no_info"
df_small[["EVENT","CAUSE","ACTION"]]  = df_small[["EVENT","CAUSE","ACTION"]].replace({0:1, 1:0})

# Remove certain passages which should not be in training or inference (these are duplicates that had to be manually found by a human)
values_to_remove = [3252, 33681, 6758, 10104]
df_small = df_small[~df_small['ID'].isin(values_to_remove)]
df_small


# create train and validation/test sets
train, test_val = train_test_split(df_small, test_size=0.3, random_state=10)
# do it again to get the test and validation sets (15% = 50% * 30%)
test, validation = train_test_split(test_val, test_size=0.5, random_state=10)




# Create an NLP friendly dataset
Hraf = DatasetDict(
    {'train':Dataset.from_dict(train.to_dict(orient= 'list')),
     'validation':Dataset.from_dict(validation.to_dict(orient= 'list')),
     'test':Dataset.from_dict(test.to_dict(orient= 'list'))})
Hraf


DatasetDict({
    train: Dataset({
        features: ['ID', 'passage', 'EVENT', 'CAUSE', 'ACTION'],
        num_rows: 1357
    })
    validation: Dataset({
        features: ['ID', 'passage', 'EVENT', 'CAUSE', 'ACTION'],
        num_rows: 291
    })
    test: Dataset({
        features: ['ID', 'passage', 'EVENT', 'CAUSE', 'ACTION'],
        num_rows: 291
    })
})

### Save partitioned dataset 

In [50]:
def make_dir(path):
    import os
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
    # Create a new directory because it does not exist
        os.makedirs(path)

# make folder if it does not exist yet
path = os.getcwd() + '/Datasets'
make_dir(path)
# save to Json
for key in Hraf.keys():
    Hraf_dict = Hraf[key].to_dict()
    with open(f"{path}/{key}_dataset.json", "w") as outfile:
        json.dump(Hraf_dict, outfile)

Make sure the training set is as biased as our groups (we want to train on as or less biased data as the groups they come from) <br>
We are shooting for .85, .68 and .68 for EVENT, CAUSE, and ACTION respectively


In [51]:
# extract the total proportion
def totalProportion(df, col, present=1):
    value_counts = df[col].value_counts()
    percentage = round(value_counts[present]/len(df)*100,2)
    return percentage

# extracts percentages per datafaframe
def colProportion(Hraf, col):
    percentage_list = []
    for dataframe in Hraf.keys():
        percentage_list += [round(sum(Hraf[dataframe][col]) / (len(Hraf[dataframe]))*100,2)]
    return percentage_list



# print bias per label
dataframe_keys= Hraf.keys()
labels = [label for label in Hraf['train'].features.keys() if label not in ['ID', 'passage']]
header = "                                TOTAL"
for key in dataframe_keys:
    header += f"     {key}"
print(header)
print('_'*(len(header)+4))
for col in labels:
    totalPercentage =  totalProportion(df_small, col)
    percentage_list =  colProportion(Hraf, col)
    spacing = 10
    percentage_str = f"{totalPercentage}{' '* (spacing-len(str(totalPercentage)))}"
    for index, key in enumerate(dataframe_keys):
        percentage_str += f"{(len(key)-5)*' '}{percentage_list[index]}{' '* (spacing-len(str(percentage_list[index])))}"
    print(f"{col}:{' ' * (30- len(col))} {percentage_str}")

                                TOTAL     train     validation     test
___________________________________________________________________________
EVENT:                          85.35     85.7           84.54     84.54     
CAUSE:                          68.18     69.42          66.32     64.26     
ACTION:                         68.44     68.39          70.45     66.67     


## Preprocess

Create labels for training and preprocessing

In [52]:

labels = [label for label in Hraf['train'].features.keys() if label not in ['ID', 'passage']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
id2label

{0: 'EVENT', 1: 'CAUSE', 2: 'ACTION'}

load a DistilBERT tokenizer to preprocess the text field: <br>

Create a preprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input length:<br>
Guidelines were followed from NielsRogge found <a href= "https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb"> here </a>

In [83]:
from transformers import AutoTokenizer
import numpy as np


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["passage"]
  # encode them
  encoding = tokenizer(text, max_length=512, truncation=True) #max length for BERT is 512
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

loading configuration file config.json from cache at /Users/ericchantland/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/ericchantland/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/vocab.txt
loading file tokenizer.json from cache at /Users/ericchantland/.cache/huggingfa

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map function. You can speed up map by setting batched=True to process multiple elements of the dataset at once:

In [85]:
# Tokenize data, remove all columns and give new ones
tokenized_Hraf = Hraf.map(preprocess_data, batched=True, remove_columns=Hraf['train'].column_names)

Map:   0%|          | 0/1357 [00:00<?, ? examples/s]

Map:   0%|          | 0/291 [00:00<?, ? examples/s]

Map:   0%|          | 0/291 [00:00<?, ? examples/s]

In [86]:
tokenized_Hraf

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1357
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 291
    })
})

In [87]:
example = tokenized_Hraf['train'][1]
print(example.keys())
print(tokenizer.decode(example['input_ids']))


dict_keys(['input_ids', 'attention_mask', 'labels'])
[CLS] in the strictest sense of the word, kwak tan is the designation for feeling pain, suffering, being indisposed, being sick. but to our selk'nam a neat separation of the physical from the psychic pain matters very little : all suffering, especially the latter, causes him an unpleasant psychic condition, and he designates predominantly this discomfort, being depressed, being indisposed psychically, with kwak tan or sickness. the more comprehensive term kwake is more in general use. the latter, however, considered as effect, in the above association of thought 756 means the same as kwak tan. ( 25 ) other feuerland / tierra del fuego / travelers emphasize this abstract distinction little or not at all. cf. beauvoir ( b ) : 48, 127 and tonelli : 116. the professional activity of a n quite generally is called kso m en = extracting the kwake from a suffering person or inflicting such on someone, sending out a nem or a hmen, etc. katmet

In [88]:
print(example['labels'])
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

[1.0, 0.0, 1.0]


['EVENT', 'ACTION']

In [92]:
# Number of passages longer than 512 tokens (and therefore truncated)
sequence_i = []
for i, tx in enumerate(tokenized_Hraf['train']):
    if len(tx['input_ids']) >= 512:
        sequence_i.append(i)
print('Number Truncated: ', len(sequence_i))
print(f'Percentage Truncated: {round(len(sequence_i)/len(tokenized_Hraf["train"])*100,1)}%')
print(sequence_i)

Number Truncated:  91
Percentage Truncated: 6.7%
[50, 55, 66, 85, 96, 97, 111, 127, 129, 143, 160, 161, 166, 175, 178, 196, 198, 212, 220, 241, 262, 268, 269, 273, 310, 346, 347, 408, 409, 416, 436, 468, 469, 477, 486, 498, 507, 570, 583, 603, 642, 643, 648, 658, 669, 674, 680, 682, 694, 718, 731, 743, 745, 756, 758, 766, 808, 841, 851, 878, 886, 938, 947, 952, 994, 1001, 1024, 1042, 1043, 1061, 1085, 1099, 1122, 1130, 1155, 1187, 1203, 1223, 1232, 1243, 1271, 1291, 1294, 1301, 1302, 1303, 1312, 1320, 1333, 1335, 1337]


In [99]:
print(len(Hraf['train'][50]['passage'].split()))
print(len(tokenized_Hraf['train'][50]['input_ids']))

571
677


Now create a batch of examples using <a href="https://huggingface.co/docs/transformers/v4.29.0/en/main_classes/data_collator#transformers.DataCollatorWithPadding"> DataCollatorWithPadding</a>. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [62]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Set tokenized passages to PyTorch Tensor

In [63]:
tokenized_Hraf.set_format("torch")

## Evaluate

Obtain F1 score for evaluation

In [64]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result


### Train
Before you start training your model, create a map of the expected ids to their labels with id2label and label2id:

In [66]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    problem_type='multi_label_classification',
    num_labels = len(labels), 
    id2label=id2label, 
    label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [67]:
#forward pass test
outputs = model(input_ids=tokenized_Hraf['train']['input_ids'][0].unsqueeze(0), labels=tokenized_Hraf['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7008, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0055, -0.0858,  0.0480]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [68]:
training_args = TrainingArguments(
    output_dir="HRAF_Model_MultiLabel_ThreeLargeClasses2",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_Hraf["train"],
    eval_dataset=tokenized_Hraf["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [69]:
trainer.train()

/opt/anaconda3/envs/NLP-3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1357
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 850
  Number of trainable parameters = 66955779


  0%|          | 0/850 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: The size of tensor a (824) must match the size of tensor b (512) at non-singleton dimension 1

In [86]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 350
  Batch size = 8


  0%|          | 0/44 [00:00<?, ?it/s]

{'eval_loss': 0.3901987075805664,
 'eval_f1': 0.8975609756097561,
 'eval_roc_auc': 0.7456310303406052,
 'eval_accuracy': 0.6142857142857143,
 'eval_runtime': 97.1293,
 'eval_samples_per_second': 3.603,
 'eval_steps_per_second': 0.453,
 'epoch': 5.0}

## Dummy Inference

In [98]:

text = "If one falls sick or dies, the natives at once conclude he must have been bewitched, or bitten, or hurt by the devil-- eringa ."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [94]:

logits = outputs.logits
logits.shape

torch.Size([1, 3])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [97]:

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['EVENT', 'CAUSE']
